In [2]:
import pandas as pd
import os
import json
from tqdm import tqdm
import glob

In [7]:

conditions = pd.read_csv('/Users/brookegrantham/Documents/DSP/anonymized_user_info_by_chunk.csv')
df_conditions = pd.DataFrame(conditions)
filenames = glob.glob('/Users/brookegrantham/Documents/DSP/shared_task_data/*/*.tweets')

In [ ]:
# count = 0
#
# for _ in tqdm(filenames):
#     with open(_, encoding='utf-8') as file:
#
#         temp_df = pd.DataFrame(json.loads(file.readline()))
#         condition = df_conditions.loc[df_conditions['anonymized_screen_name'] == temp_df['user']['screen_name']]['condition'].item()
#         username = temp_df['user']['screen_name']
#
#         for line in file:
#             try:
#                 data = pd.DataFrame(json.loads(line))
#                 tweets = pd.concat([tweets, pd.DataFrame({'text':[data['text']['screen_name']],
#                                                           'username': [username],
#                                                           'condition': [condition]})])
#             except:
#                 print(line)
#                 count+=1
#                 pass
# print('Total number of dropped lines is:', count)

In [15]:
tweets_dir = '/Users/brookegrantham/Documents/DSP/shared_task_data/'

for i in tqdm(range(60,90)):
    filenames = glob.glob(f'{tweets_dir}/{i}/*.tweets')

    all_tweets = []

    for filename in filenames:
        with open(filename, encoding='utf-8', errors='replace') as f:
            json_lines = f.readlines()
            all_tweets += [{'filename': filename, **json.loads(x)} for x in json_lines]

    tweets_df = pd.DataFrame(all_tweets)
    tweets_df['username'] = tweets_df['user'].apply(lambda d: d.get('screen_name'))

    tweets_df = tweets_df.merge(df_conditions[['anonymized_screen_name', 'condition']], left_on='username', right_on='anonymized_screen_name', how='left')
    tweets_df = tweets_df[['username', 'condition', 'text']]
    tweets_df.to_csv(f'{i}_conv.csv', index=False, errors='replace')


100%|██████████| 30/30 [02:42<00:00,  5.40s/it]


Haha turns out it goes training set 0-50 and test set 60-89

In [46]:
import csv
filenames2 = glob.glob('*_conv.csv')

final_tweets = []
for filename in filenames2:
    with open(filename) as f:
        reader = csv.reader(f)
        final_tweets += list(reader)

In [47]:
temp_df = pd.DataFrame(final_tweets)

In [74]:
temp_df

,username,condition,text
1,puTtjUFpRciDHX,control,RT @w3BAUZRi: Hoes be 15 looking 20.\n\nI be n...
2,puTtjUFpRciDHX,control,All i'm thinking bout is money i'll leave the ...
3,puTtjUFpRciDHX,control,@sI6UT_PskVAZ1I7 fb
4,puTtjUFpRciDHX,control,@fO3aX4BnZG fb
5,puTtjUFpRciDHX,control,@okLAE3IXeny dnt worry boug that
...,...,...,...
4034200,oiEx1JBNkiVqGa,depression,"Reba McEntire is the best!!! ""No matter what f..."
4034201,oiEx1JBNkiVqGa,depression,RT @nD_h: It keeps getting better!! We got ove...
4034202,oiEx1JBNkiVqGa,depression,RT @nKX5elH1s5Iy5: Consider me a smile in an o...
4034203,oiEx1JBNkiVqGa,depression,RT @b9IuO9i9Yxzz: Hey guys can you please retw...


In [60]:
temp_df = temp_df.rename(columns={0:'username', 1:'condition', 2:'text'})

In [67]:
temp_df = temp_df[temp_df.username!='username']

In [76]:
print( 'Number of users with depression:', len(temp_df[temp_df.condition=='depression']))
print( 'Number of users with ptsd:', len(temp_df[temp_df.condition=='ptsd']))
print( 'Number of control:', len(temp_df[temp_df.condition=='control']))

Number of users with depression: 1132270
Number of users with ptsd: 919368
Number of control: 1978143


In [4]:
temp_df = pd.read_csv('all_tweets')

In [12]:
temp_df = temp_df[['username', 'condition', 'text']]

In [13]:
temp_df.to_csv('all_tweets', index=False, errors='replace')

In [16]:
from sklearn.model_selection import train_test_split

training_tweets, temp_tweets, training_labels, temp_labels = train_test_split(temp_df['text'],temp_df['condition'], test_size=0.3, random_state=123)
val_tweets, test_tweets,val_labels, test_labels = train_test_split(temp_tweets, temp_labels, test_size=(1/3), random_state=123)